# Intro to Argovis' Grid API

Argovis offers a growing list of gridded products, indexed and downloadable through its API. In this notebook, we'll illustrate some basic operations and handling of this data.

> **This is a beta product!**
> Argovis' new API is currently undergoing heavy development. Therefore, everything in these notebooks should be understood as a technical preview only; implementation details may change before a 
stable release is made. Please send feedback and ideas to argovis@colorado.edu, and see the API docs as they evolve at https://argovis-api.colorado.edu/docs/.

## Setup

In addition to importing a few python packages, make sure to plug in your Argovis API key for `API_KEY` in the next cell. If you don't have a free Argovis API key yet, get one at https://argovis-keygen.colorado.edu/.

In [1]:
import requests, xarray, pandas, math, datetime, copy
import numpy as np
from datetime import datetime, timedelta

API_KEY=''
#API_PREFIX = 'https://argovis-api.colorado.edu'
API_PREFIX = 'https://argovis-apix-atoc-argovis-dev.apps.containers02.colorado.edu'

## Downloading Gridded Data

Argovis supports a number of grids through the same API endpoints. Let's begin by discovering what grids are available:

In [2]:
grids = requests.get(API_PREFIX + '/grids/vocabulary', headers={'x-argokey': API_KEY}).json()
grids

{'code': 429,
 'message': 'You have temporarily exceeded your API request limit. You will be able to issue another request in NaN seconds. Long term, requests like the one you just made can be made every 1 seconds.'}

We get a list of grid names from the `/grids/vocabulary` route. We can query data from any one of these via the `/grids/{gridName}` route, and the usual temporospatial parameters that may be familiar from other Argovis APIs:

 - `startDate` (geo-constrained, see below, format YYYY-MM-DDTHH:MM:SSZ at GMT0): beginning of time window to query.
 - `endDate` (geo-constrained, see below, format YYYY-MM-DDTHH:MM:SSZ at GMT0): end of time window to query.
 - `polygon` (geo-constrained, see below, format [[lon0,lat0],[lon1,lat1],...,[lonN,latN],[lon0,lat0]]): geographical region to query.
 - `multipolygon` (geo-constrained, see below, format as a list of `polygon`s): query the geographical region at the intersection of *all* listed polygons.
 - `center` + `radius` (geo-constrained, see below, center format lon,lat ; raidus format distance in km): geographical region to query, defined as maximum distance (radius) from a point (center).
 - `presRange` (optional, format shallowlimit,deeplimit): only return levels between the limits specified, in dbar.
 - `data` (optional, format comma separated list of keys): request the actual measurements to go with matched documents.
 
**Items marked 'geo-constrained'** are individually optional, but have soft limits based on estimates of how much data they will return. If you need to query a large area, consider making requests over a short timespan; or vice versa.

Let's try a simple request to download a piece of data from one of these gruds, a 10 degree box over the North Atlantic from the first quarter of 2012 from the Roemmich-Gilson Argo temperature climatology. For gridded data, the `data` key, when used, always matches the grid name in the route.

In [3]:
params = {
  "startDate": '2012-01-01T00:00:00Z',
  "endDate": '2012-04-01T00:00:00Z',
  "polygon": '[[-66.621094,42.163403],[-71.367188,40.580585],[-74.003906,37.439974],[-75.058594,35.029996],[-79.453125,32.249974],[-80.15625,29.993002],[-78.75,23.725012],[-73.828125,21.289374],[-70.3125,20.797201],[-67.5,19.47695],[-64.160156,19.47695],[-66.621094,42.163403]]',
  "data": 'temperature_rg'
}

rgdata = requests.get(API_PREFIX + '/grids/temperature_rg', params=params, headers={'x-argokey': API_KEY}).json()

(If you need help constructing a polygon, try the following:

  - visit argovis.colorado.edu
  - draw a shape
  - click on the purple shaded area of the region of interest (not on a dot)
  - from the pop up window, go "to Selection page"
  - from the url of the selection shape, copy the shape, i.e. [copy_all_this_inside_outer_brackets] after 'shape=')


Like most Argovis API requests, you get a list of documents matching your query. Let's have a look at the first record in what the API returned to us:

In [4]:
rgdata[0]

KeyError: 0

By default, Argovis returns gridded data in a *profile-like* structure, with a schema similar to other Argovis APIs. Data is presented in the `data` key, with units for each key presented in the `units` property. Since a grid reports values at regular depths, those depths are reported in the metdata document associated with each grid profile in order to avoid trasmitting them over and over for default requests like this. We can look them up using the `/grids/metadata` route and the `metadata` key from the document in question:

In [ ]:
metadata_params = {
    "id": rgdata[0]['metadata']
}

rgmeta = requests.get(API_PREFIX + '/grids/meta', params=metadata_params, headers={'x-argokey': API_KEY}).json()
rgmeta

Here we see some interesting metadata about the file that produced the first record in our list of grid estimates, including the levels by which we can interpret the `data` key: these lists are sorted identically, so the first element in the `data` list corresponds to a depth of 2.5 dbar, the first entry in the `levels` metadata key.

If we're only interested in the grid estimates near the ocean surface, we can add a `presRange` filter to our data request:

In [ ]:
filter_params = copy.deepcopy(params)
filter_params['presRange'] = '0,100'

rgSurfacedata = requests.get(API_PREFIX+'/grids/temperature_rg', params=filter_params, headers={'x-argokey': API_KEY}).json()
rgSurfacedata[0]

Now our data records returned by the `grids/temperature_rg` route only report levels within the range requested; also note when we customize the depth range requested, the custom-filtered list of levels appears directly on the data document. **When a key, such as levels, appears on a data document and that document's corresponding metadata document, the assigned value on the data document always takes precedence**.

When downloading large volumes of data, we can also request that it be minified:

In [ ]:
mini_params = copy.deepcopy(params)
mini_params['compression'] = 'basic'

rgdata = requests.get(API_PREFIX+'/grids/temperature_rg', params=mini_params, headers={'x-argokey': API_KEY}).json()
rgdata[0]

## Ingestion by xarray

Xarray is a familiar pythonic data structure; we can transform a raw API response to an xarray with a helper similar to the following.

In [ ]:
def xargrid(grid, depths):
    # given the json response <grid> of a request to /grids/{gridName},
    # a list <depths> of the corresponding depths for these grid documents
    # return an xarray object with coordinates time, lat, lon, depth, and measurement value.
    
    lat = []
    lon = []
    time = []
    meas = []
    pressure = []
    for p in grid:
        for i, e in enumerate(p['data']):
            lon.append(p['geolocation']['coordinates'][0])
            lat.append(p['geolocation']['coordinates'][1])
            # convert a string to a date (zeros to check at the end of date)
            time.append(datetime.strptime(p['timestamp'], '%Y-%m-%dT%H:%M:%S.%fZ'))
            meas.append(p['data'][i][0])
            pressure.append(depths[i])
            
    df = pandas.DataFrame({"latitude": lat, 
                           "longitude": lon, 
                           "time": time, 
                           "pressure": pressure, 
                           "measurement": meas}).set_index(["latitude","longitude","time","pressure"])
    return df.to_xarray()
    
ds = xargrid(rgdata, rgmeta[0]['levels'])

Now we can do all the usual xarray operations; lets see what the ranges of our coordinate variables are:

In [ ]:
print('latitudes:',ds['latitude'].data)
print('longitudes:',ds['longitude'].data)
print('times:',ds['time'].data)
print('pressures:',ds['pressure'].data)

We can easily select a slice of this array at constant pressure and time, to produce a possibly more conventional, map-like grid representation, and plot it with xarray's built in plots:

In [ ]:
gridmap = ds.loc[{"time":'2012-01-15T00:00:00.000Z', "pressure":2.5}]
gridmap['measurement'].plot()

## Area-Weighted Means over a longitude/latitude region

A common operation when considering gridded data is to weight a mean by area of grid cells, which changes with latitude. A helper to do this with Argovis grid data could look like the following.

In [ ]:
def areaweighted_region_mean(dxr):
    # given an xarray dataset <grid> for a given depth and time,
    # calculate the mean of the gridded data variable, weighted by grid cell area
    weights = np.cos(np.deg2rad(dxr.latitude))
    weights.name = "weights"
    dxr_weighted = dxr.weighted(weights)
    
    return dxr_weighted.mean(("longitude", "latitude"))

In [ ]:
dxr_aw = areaweighted_region_mean(dxr=ds)
print(dxr_aw.loc[{"time":'2012-01-15T00:00:00.000Z', "pressure":2.5}])
dxr_aw

In [ ]:
# Let's plot the data after the spatial average over longitude and latitude
dxr_aw['measurement'].plot(y="pressure",yincrease=False,aspect=0.5, size=10)

In [ ]:
# Let's plot the data again, this time plotting one line per timestep
dxr_aw['measurement'].plot.line(y="pressure",yincrease=False,aspect=1, size=10)

## Accessing and visualizing Ocean Heat Content (OHC)

OHC fields in the following are by Kuusela and Giglio 2022 (https://doi.org/10.5281/zenodo.6131625) and are mapped using locally stationary Gaussian processes with data-driven decorrelation scales (Kuusela and Stein, 2018). A linear time trend was included in the estimate of the mean field (along with spatial terms and harmonics for the annual cycle). Mapping is done in latitude and longitude with monthly subsets of data for e.g. the 15-300 dbar pressure layer. The top layer of the pressure range is indicated in "levels".

In [ ]:
params = {
  "startDate": '2012-01-01T00:00:00Z',
  "endDate": '2013-01-01T00:00:00Z',
  "polygon": '[[-78,30],[-70,30],[-70,20],[-78,20],[-78,30]]',
  "data": 'ohc_kg',
  "compression": "basic"
}

ohc_kg = requests.get('https://argovis-api.colorado.edu/grids/ohc_kg', params=params, headers={'x-argokey': API_KEY}).json()

metadata_params = {
    "id": ohc_kg[0]['metadata']
}

ohcmeta = requests.get('https://argovis-api.colorado.edu/grids/meta', params=metadata_params, headers={'x-argokey': API_KEY}).json()

In [ ]:
ds_ohc = xargrid(ohc_kg, ohcmeta[0]['levels'])

In [ ]:
ds_ohc

Let's plot a map of OHC for January 2012:

In [ ]:
gridmap = ds_ohc.loc[{"time":'2012-01-15T00:00:00.000Z',"pressure":15}]
gridmap['measurement'].plot()

Let's now compute the area average in the selected region.

In [ ]:
dxr_aw_ohc = areaweighted_region_mean(dxr=ds_ohc)
dxr_aw_ohc['measurement'].plot()